<a href="https://colab.research.google.com/github/djili/data-processing/blob/main/ConnectDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connection a une base de donnees

Apprendre à se connecter à une base de données PostgreSQL, lire des données depuis une table, exécuter des requêtes SQL et écrire des résultats dans une nouvelle table en utilisant Spark SQL.

1. Créez une session Spark pour initialiser l'environnement de travail.
2. Configurez les paramètres de connexion JDBC pour se connecter à PostgreSQL.
3. Lisez les données d'une table existante dans la base de données PostgreSQL et affichez.
4. Exécutez quelques requêtes SQL pour transformer et analyser les données.
5. Écrivez les résultats des transformations dans une nouvelle table de la base de données PostgreSQL.


In [3]:
from pyspark.sql import SparkSession

In [4]:
# Créer une session Spark avec le driver JDBC PostgreSQL
spark = SparkSession.builder.appName("PostgresSparkSQL").config("spark.jars", "/content/postgresql-42.7.7.jar").getOrCreate()


In [5]:
# 2. Paramètres de connexion JDBC
jdbc_url = "jdbc:postgresql://ep-silent-water-a2w91dtv.us-east-2.aws.neon.tech/sample_db"
connection_properties = {
    "user": "sample_user",
    "password": "sample_password",
    "driver": "org.postgresql.Driver"
}

In [6]:
# 3. Lire les données d'une table PostgreSQL
df = spark.read.jdbc(url=jdbc_url, table="public.countries", properties=connection_properties)
print("✅ Données initiales :")
df.show(5)

Py4JJavaError: An error occurred while calling o28.jdbc.
: org.postgresql.util.PSQLException: ERROR: password authentication failed for user 'sample_user'
	at org.postgresql.core.v3.ConnectionFactoryImpl.doAuthentication(ConnectionFactoryImpl.java:777)
	at org.postgresql.core.v3.ConnectionFactoryImpl.tryConnect(ConnectionFactoryImpl.java:234)
	at org.postgresql.core.v3.ConnectionFactoryImpl.openConnectionImpl(ConnectionFactoryImpl.java:289)
	at org.postgresql.core.ConnectionFactory.openConnection(ConnectionFactory.java:57)
	at org.postgresql.jdbc.PgConnection.<init>(PgConnection.java:277)
	at org.postgresql.Driver.makeConnection(Driver.java:448)
	at org.postgresql.Driver.connect(Driver.java:298)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.BasicConnectionProvider.getConnection(BasicConnectionProvider.scala:49)
	at org.apache.spark.sql.execution.datasources.jdbc.connection.ConnectionProviderBase.create(ConnectionProvider.scala:102)
	at org.apache.spark.sql.jdbc.JdbcDia

In [ ]:
# 4. Créer une vue temporaire pour faire des requêtes SQL
df.createOrReplaceTempView("countries")

In [ ]:
# 5. Requête SQL d'analyse (exemple : filtrer les pays d’Afrique)
df_afrique = spark.sql("""
    SELECT name, region
    FROM countries
    WHERE region = 'Africa'
""")
print("🌍 Pays d'Afrique :")
df_afrique.show()

In [ ]:
# 6. Écrire les résultats dans une nouvelle table PostgreSQL
df_afrique.write.jdbc(
    url=jdbc_url,
    table="public.african_countries",
    mode="overwrite",  # ou "append"
    properties=connection_properties
)

print("✅ Données écrites dans la table 'african_countries'